# Chicago Car Crashes Project

## Business Understanding

### One of the purposes of having vehicle crash data is to; Better public safety, Improve urban planning, and Improve policy making

### Some of the possible business questions that can be derived from the data include, 

#### 1. What factors contribute most to severe crashes?
#### 2. Which locations, times, and conditions are accident prone?
#### 3. Are certain groups more vulnerable to crashes?
#### 4. Which vehcile types are most involved in severe or fatal crashes?
#### 5. How do some behaviors impact crashes e.g seatbelt use, intoxication, or distractions affect injury severity?
#### 6. How can the data being assessed be used to assist the police, hospitals, and city planners target interventions? 

## Problem Statement
#### Pinpoint crash hotspots in Chicago and understand contributing factors to assists city planners and law enforcement to minimize accidents.

## Metric for Success

### Sucessfully answering the above business questions will be a a significant advantage.
### Another metric will be making a hotspot analysis that accurately pinpoints high_risk zones for crashes. 
## Real World Use Case
### could be in assisting governments and city planners on the regions that they need to install cameras, improve lighting, and redesign road structures in hotspots.


### Considering that lives are involved and this is my first official model, an accuracy of 80% will be considered sufficient.

## Data Understanding



In [ ]:
#import the libraries
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

#import sklearn libraries
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, SMOTEN
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report,roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

In [ ]:
# load the datasets
people_data = pd.read_csv("Traffic_Crashes_People.csv", low_memory=False)
vehicles_data = pd.read_csv("Traffic_Crashes_Vehicles.csv", low_memory=False)
crashes_data = pd.read_csv("Traffic_Crashes_Crashes.csv", low_memory=False)

In [ ]:
# checking the people dataset
people_data.head()

In [ ]:
# checking the vehicles dataset
vehicles_data.head()

In [ ]:
# checking the vehicles dataset
crashes_data.head()

In [ ]:
# checking the shape on both datasets
print(f"The people dataset has a shape of {people_data.shape}, the vehicles dataset has a shape of {vehicles_data.shape}, the crashes dataset has a shape of {crashes_data.shape}.")

In [ ]:
# Checking the column names in both datasets
print(people_data.columns)

print(vehicles_data.columns)

print(crashes_data.columns)

#### From the above, the vehicles dataset has a lot more columns than the peoples dataset. 
#### However, I will still try to merge the two datasets so that I can start working from a single dataset.

In [ ]:
# The approach for joining the two datasets will be to join on the crash record ID and vehicle ID column since they are similar in both sets.
# This approach will help in minimizing any cases of duplicates

# The first part of this approach is to standardise the columns to avoid any mismatches that may occur
people_data.columns = people_data.columns.str.lower()
vehicles_data.columns = vehicles_data.columns.str.lower() 
crashes_data.columns = crashes_data.columns.str.lower()
# Next, merging on crash record ID and vehcile ID columns


# Disclaimer, as a result of the meagre computing power on my pc, I decided to go with inner merge. 
people_vehicles = pd.merge(
    people_data,
    vehicles_data,
    how="inner",
    on=["crash_record_id", "vehicle_id"]
)

# Merging the third dataset
merged_df = pd.merge(    
    people_vehicles,
    crashes_data,
    how="inner",
    on="crash_record_id"
)


print (merged_df.shape)

In [ ]:
# Checking the first 5 columns on the merged data
merged_df.head()

In [ ]:
merged_df.columns

In [ ]:
# Checking the percentage of missing data
missing_data = merged_df.isnull().mean()*100

#  Sorting the missing data in descending order
missing_data = missing_data.sort_values(ascending=False)

print(missing_data)

In [ ]:
# Now considering the number of rows and columns, I will be dropping the columns with more than 70% missing data. 
# The 70% figure is based on the fact that there is too much missing data for that column to be useful.

# Threshold (70%)
threshold = 0.7

# Drop columns where more than 70% values are missing
merged_df = merged_df.loc[:, merged_df.isnull().mean() < threshold]

# Checking the percentage of missing data
missing_data = merged_df.isnull().mean()*100

#  Sorting the missing data in descending order
missing_data = missing_data.sort_values(ascending=False)

print(missing_data)

In [ ]:
# Checking the data's new shape
merged_df.shape

In [ ]:
merged_df.head()

In [ ]:
# Print all column names to check for typos or different names
print(merged_df.columns)

In [ ]:
# Now, for the remaining columns, I will try to use relationships and columns to fill in the missing data

corrs = merged_df.corr(numeric_only=True)["num_passengers"].sort_values(ascending=False)

print(corrs) 

In [ ]:
# Since there is a high correlation with occupant_cnt, I will start by filling in that column, which will the be used to fill in the num_passengers column

# Filling in occupant_cnt using mode
mode_value = merged_df["occupant_cnt"].mode()[0]

merged_df["occupant_cnt"] = merged_df["occupant_cnt"].fillna(mode_value)

# check if it took
merged_df["occupant_cnt"].isnull().sum()

In [ ]:
# using the occupant cnt column to fill in the num passengers

merged_df["num_passengers"] = merged_df.apply(
    lambda row: row["occupant_cnt"] - 1
    if pd.isna(row["num_passengers"]) else row["num_passengers"], 
    axis=1
)

# check if it took 
merged_df["num_passengers"].isnull().sum() 

In [ ]:
# Filling in the data from columns
columns_to_fill_with_mode = [
    "drivers_license_class", "drivers_license_state", "zipcode", "age","city", "state", "driver_vision", "driver_action", "bac_result",
    "physical_condition", "vehicle_year", "lic_plate_state", "first_contact_point","model", "make", "occupant_cnt", "maneuver", "travel_direction",
    "vehicle_use", "vehicle_type", "vehicle_defect", "vehicle_id","airbag_deployed", "sex", "ejection", "safety_equipment", "injury_classification",
    "unit_type","report_type","location","longitude","latitude","most_severe_injury","beat_of_occurrence","street_direction","street_name" 
]

# Looping through and filling each with the mode 

for col in columns_to_fill_with_mode:
    mode_value = merged_df[col].mode()[0]   # get most common value
    merged_df[col] = merged_df[col].fillna(mode_value)

# Check one of them
print("Missing drivers_license_class:", merged_df["drivers_license_class"].isnull().sum())

In [ ]:
# checking for columns in missing data
missing_data2 = merged_df.isnull().mean()*100

#  Sorting the missing data in descending order
missing_data2 = missing_data2.sort_values(ascending=False)

print(missing_data2)


In [ ]:
#confirm the imputation
merged_df.isna().sum().any()

In [ ]:
#check duplicates
merged_df.duplicated().sum()

# Data Preparation


In [ ]:
# Making a copy of the cleaned dataset

cleaned_data = merged_df.copy(deep=True)
cleaned_data.shape

In [ ]:
cleaned_data.columns

In [ ]:
cleaned_data.head()

In [ ]:
# checking for outliers
sns.boxplot(cleaned_data,color="r")
plt.tight_layout()
plt.grid(alpha=.3)
plt.xticks(rotation=45)
plt.show();

From the plot above, the vehicle_Id and crash_unit are unique identifiers assigned to each record 

# Exploratory Data Analysis


In [ ]:
# Considering the size of the dataset, the next step will be to get a sample of 10% of the data of randomly selected rows to use for analysis
sample_data = cleaned_data.sample(frac=0.1, random_state=42)
sample_data.shape


##### The above creates a sample size of 200K which allows me to conduct my analysis in a way that my P.C can handle.

In [ ]:
sample_data.head()

In [ ]:
# Considering my problem statement and maintaining the same, EDA will focus along those lines

# Making sure that the 'latitude' and 'longitude' columns do not have any unfilled sections
df_spatial = sample_data.dropna(subset=['latitude', 'longitude'])

# To make sure that both the latitude and longitude only cover the Chicago region
df_spatial = df_spatial[
    (df_spatial['latitude'].between(41.6, 42.1)) &
    (df_spatial['longitude'].between(-87.95, -87.5))
]

print(f"Remaining rows after filtering: {len(df_spatial)}")

In [ ]:
# Static density visualization which shows a static heatmap where the crashes are concentrated

plt.figure(figsize=(8, 8))
plt.hexbin(df_spatial['longitude'], df_spatial['latitude'], gridsize=100, cmap='Reds', bins='log')
plt.colorbar(label='log(crash count)')
plt.title("Crash Density Across Chicago")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
# Hotspot clustering using DBSCAN
from sklearn.cluster import DBSCAN
coords = df_spatial[['latitude','longitude']].to_numpy()

# scaling for DBSCAN
coords_scaled = StandardScaler().fit_transform(coords)

db = DBSCAN(eps=0.05, min_samples=30).fit(coords_scaled)  
df_spatial['cluster'] = db.labels_

print(df_spatial['cluster'].value_counts().head())

In [ ]:
# Ranking the hotspots
hotspots = df_spatial.groupby('cluster').agg(
    crashes=('crash_record_id', 'count'),
    total_injuries=('injuries_total','sum'),
    fatalities=('injuries_fatal','sum')
).sort_values('crashes', ascending=False)

# Checking the order of the hostspots
hotspots.head(10)

In [ ]:
# Visualizing the cluster on a map with the crashes being color-coded according to the clusters
import folium
m_clusters = folium.Map(location=[41.85, -87.65], zoom_start=11)

for _, row in df_spatial.iterrows():
    if row['cluster'] != -1:
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=2,
            color=f"#{(hash(row['cluster']) & 0xFFFFFF):06x}",  # cluster color
            fill=True,
            fill_opacity=0.6
        ).add_to(m_clusters)

m_clusters

# Univariate Analysis

In [ ]:
# These lines of code are meant to structure the relevant variables according to the hotspot issues instead of brutforcing all the columns 
# and data which are quite substantial

# Detailing the appropriate code for making the necessary plots for the rest of the workflow.

def plot_categorical(series, top_n=10):
    counts = series.value_counts().head(top_n)
    sns.barplot(x=counts.values, y=counts.index)
    plt.title(series.name)
    plt.show();

def plot_numeric(series, bins=20):
    sns.histplot(series.dropna(), bins=bins, kde=False)
    plt.title(series.name)
    plt.show();


In [ ]:
# Getting the time when ost crashes occur
plot_numeric(sample_data['crash_hour'])


From the above, the distribution is according to the period or hour when most crashes tend to occur with most of the crashes happening at 3pm followed by 7 am, and around midnight. This information can help us better explain some of the reasons being the 3pm jam that occurs as most people are leaving work, conducting their errands, and picking children up from school. The 7AM crashes can be explained as the early morning bustle as people try to go to work and school.

In [ ]:
# Map the days of the week if stored as numbers
day_map = {
    1: "Monday", 2: "Tuesday", 3: "Wednesday",
    4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"
}
sample_data['crash_day_of_week'] = sample_data['crash_day_of_week'].map(day_map)

# Ensure categorical order
days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
sample_data['crash_day_of_week'] = pd.Categorical(
    sample_data['crash_day_of_week'],
    categories=days_order,
    ordered=True
)

# Plot
sns.countplot(data=sample_data, y='crash_day_of_week', order=days_order)
plt.title("Crashes by Day of Week")
plt.show()


In [ ]:
# Month map: 1-12 → Month names
month_map = {
    1: "January", 2: "February", 3: "March", 4: "April",
    5: "May", 6: "June", 7: "July", 8: "August",
    9: "September", 10: "October", 11: "November", 12: "December"
}

# Apply mapping
sample_data['crash_month'] = sample_data['crash_month'].map(month_map)

# Chronological order (Jan → Dec)
month_order = list(month_map.values())

# Plot with order
sns.countplot(
    data=merged_df,
    x='crash_month',
    order=month_order,
    palette="crest"
)
plt.title("Crashes by Month")
plt.xlabel("Month")
plt.ylabel("Number of Crashes")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Consideirng the deverity of the crashes
# The most severe injuries
plot_categorical(sample_data['most_severe_injury'])

In [ ]:
plot_categorical(sample_data['damage'])

In [ ]:
# An assessment of the weather conditions and when most accidents occured

plot_categorical(merged_df['weather_condition'])

In [ ]:
# An assessment of the lighting conditions when most accidents occured

plot_categorical(merged_df['lighting_condition'])

In [ ]:
# The existence of traffic controls whenever accidents occurred

plot_categorical(merged_df['traffic_control_device'])

In [ ]:
# The road surface conditions whenever accidents occured and when most accidents occurred

plot_categorical(merged_df['roadway_surface_cond'])

In [ ]:


plot_categorical(merged_df['prim_contributory_cause'])

In [ ]:
# An assessment of the gender that led to the most accidents

plot_categorical(merged_df['sex'])

In [ ]:
# Getting an assessment of the ages of people involved in the accidents

plot_numeric(merged_df[merged_df['age'] > 0]['age'], bins=10)

In [ ]:
# Instances when the people involved in accidents had worn safety equipment

plot_categorical(merged_df['safety_equipment'])

In [ ]:
# Whether or not BAC tests were administered

plot_categorical(merged_df['bac_result'])

In [ ]:
# The type of vehicle involved in an accident

plot_categorical(merged_df['vehicle_type'])

In [ ]:
# Whether or not the vehicle had any defects

plot_categorical(merged_df['vehicle_defect'])

# Bivariate Analysis

## An assessment of the relationship between features.
### For this section I will majorly focus on elements that majorly align with my problem statement.

In [ ]:
# Age vs injury classification 

plt.figure(figsize=(10,6))
sns.boxplot(data=sample_data, x='injury_classification', y='age')
plt.title("Driver Age vs Injury Classification")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# An assessment of the vehicle type vs injury classification
plt.figure(figsize=(12,6))
sns.countplot(data=sample_data, x='vehicle_type', hue='injury_classification',
              order=sample_data['vehicle_type'].value_counts().iloc[:10].index)
plt.xticks(rotation=45)
plt.title("Vehicle Type vs Injury Classification")
plt.show()

In [ ]:
# An assessment of the relationship between alcoholism and injury
plt.figure(figsize=(8,5))
sns.countplot(data=sample_data, x='bac_result', hue='injury_classification')
plt.title("ALCOHOL (BAC) Result vs Injury Classification")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Your crash_hour is already in the right format - just use it directly!
simple_grouped = merged_df.groupby('crash_hour').size()

# Create the plot
plt.figure(figsize=(12, 6))
simple_grouped.plot(kind='bar')
plt.title('Traffic Crashes by Hour of Day')
plt.xlabel('Hour (24-hour format)')
plt.ylabel('Number of Crashes')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Multivariate Analysis
### Interactions between multiple variables



In [ ]:
# Considering Vehicle Year, Age, and Injury

plt.figure(figsize=(12,6))
sns.scatterplot(data=sample_data, x='vehicle_year', y='age', hue='injury_classification', alpha=0.6)
plt.title("Vehicle Year vs Age colored by Injury Classification")
plt.show();

# A correlation Heatmap of the numerical features

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(sample_data[['age','num_passengers','vehicle_year','occupant_cnt']].corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap of Numerical Features")
plt.show();

# Modelling - Making A HotSpot Analysis
## Data Preprocessing

In [ ]:
# Lets start with defining the target variable

# the first step involves counting the crashes per zipcode 

zipcode_counts = cleaned_data.groupby("zipcode").size().reset_index(name="crash_count")

# Then we can define the threshold for hotspots (Top 10% zipcodes)
threshold = zipcode_counts["crash_count"].quantile(0.9)

# Creating a new hotspots column 
zipcode_counts["hotspot"] = (zipcode_counts["crash_count"] >= threshold).astype(int)

# create a new dataset for modelling
modelling_data = cleaned_data.merge(zipcode_counts[["zipcode", "hotspot"]], on="zipcode", how="left")

# preview the top ten crash_heavy zipcodes
print(zipcode_counts.sort_values("crash_count", ascending=False).head(10))

# preview the merhed modelling dataset
print(modelling_data.head())

## Feature Engineering for the first model that predicts the areas that are prone to crashes per zipcode.


In [ ]:
import pandas as pd

# make a copy of the cleaded data 
df = cleaned_data.copy()
df = df[df["zipcode"].notna()]

current_year = pd.Timestamp.now().year

features_by_zipcode = df.groupby("zipcode").agg(
    crash_count=("crash_record_id", "nunique"),               # unique crashes per zipcode
    avg_age=("age", "mean"),
    prop_male=("sex", lambda x: (x == "M").mean()),
    prop_missing_license=("drivers_license_class", lambda x: x.isna().mean()),
    prop_bac_positive=("bac_result", lambda x: (x == "Positive").mean()),
    driver_action_nunique=("driver_action", "nunique"),
    driver_vision_nunique=("driver_vision", "nunique"),
    prop_missing_physical_condition=("physical_condition", lambda x: x.isna().mean()),
    avg_vehicle_year=("vehicle_year", "mean"),
    avg_vehicle_age=("vehicle_year", lambda x: (current_year - x).mean()),
    prop_vehicle_defect=("vehicle_defect", lambda x: x.notna().mean()),
    vehicle_type_nunique=("vehicle_type", "nunique"),
    prop_airbag_deployed=("airbag_deployed", lambda x: (x == "Yes").mean()),
    avg_occupant_cnt=("occupant_cnt", "mean"),
    maneuver_nunique=("maneuver", "nunique"),
    travel_direction_nunique=("travel_direction", "nunique")
).reset_index()

# Merge hotspot labels you created earlier
modelling_data = features_by_zipcode.merge(
    zipcode_counts[["zipcode", "hotspot"]],
    on="zipcode",
    how="left"
)

print(modelling_data.head())


### Train - Test Split

In [ ]:
modelling_data.head()


In [ ]:
# Define the features and target
X = modelling_data.drop(columns=["hotspot", "zipcode"])
y = modelling_data["hotspot"]

# make the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#check shape
X_train.shape, X_test.shape, y_train.shape, y_test.shape

#### The results above shows that the feature engineering was successful since its purpose was to aggregate the 2 million person-level rows into zipcode-level features. The result was that it drastically reduced the dataset since Chicago has so many zipcodes. This will allow us to make predictions according to the existing zipcodes with the right data. 

In [ ]:
#  to check if the data columns are indeed numeric
X_train.dtypes

#### Since from the above columns, it is clearly evident that the columns are all numeric, and can then continue with the rest of the process

In [ ]:
# Since the above confirms that the columns are numeric
numeric_cols = X_train.columns

# create a pipeline that scales the numeric features + logistic regression
pipeline = Pipeline (steps=[
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

# model fitting
pipeline.fit(X_train, y_train)



In [ ]:
# checking the models performance on training data
print(f"The model's score on training data is {pipeline.score(X_train, y_train)*100:.2f}%")

In [ ]:
# Assessing the models prediction and accuracy
y_pred = pipeline.predict(X_test)

#accuracy
acc = accuracy_score(y_test, y_pred)*100

print(f"The model's accuracy on unseen data is {round(acc, 2)}%")

In [ ]:
conf = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))
plt.title("Confusion Matrix")
sns.heatmap(conf, annot=True, fmt="d")
plt.show()


#### The above confutions matrix shows on the top left the correctly predicted non-hotspots, the bottom right shows the cirrectly predicted hotspots. The bottom left and top right regions shows the existing misclassifications.

In [ ]:
# classification report
print(classification_report(y_test, y_pred))

#### The above report shows that the class 0 shows the non-hotspot zipcodes, class 1 hotspot zipcodes. For the Non-hotspots, the precision is at a hundred%, the recall is at 95% which means that it catches 955 of all true non_hotspots, an f1-score of 97% whihc translates to a strong overall balance. From the presented figures making the conclusion that the model never mislabels a hotspot as a non_hotspot would be an accurate assessment.

#### In the hotspots region, the precision score is at 70% which means that it is accurate 70% of the time with 30% of the instances being false alarms. The recall figures means that it catches about 97% of the real hotspots, the f1-score of 81% is also quite strong but quite lower to the non hotspots becaise of the drop in precision. 

#### The overall conclusion is that the model is excellent in finding hotspots, however, it has instances where it over-predicts and marks safe zipcodes as hotspots. 

# Layman's Interpretation of the model

#### The model is a binary classification model that predicts whether or not a zipcode is a crash hotspot. At this level I'm not sure whether I can predict future outcomes but I'm sure that I can classify whether an areas is a high-risk hotspot for crashes. 

## Business Interpretation 

#### The model is quite good at catching hotspots with a recall of 97%, but it still has instances where it gives false positives which is shows by the low precision score of 70% where it flags safe areas as hotspots. In the instance of the current problem statement, this isnt a bad trade-off since it may be better to over-predict a hotspot resulting in extra caution than missing a true hotspot, which will result in people getting hurt and in the owrst instances, deaths. 


# Random Forest Classifier

In [ ]:
# initializing the random forest model

rf_classifier = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    class_weight="balanced",
    random_state=42
)

# No need for another train-test split since it was already done above 

# model training
rf_classifier.fit(X_train, y_train)

# make predictions 
y_pred_train = rf_classifier.predict(X_train)
y_pred_test = rf_classifier.predict(X_test)

print(classification_report(y_train, y_pred_train))



In [ ]:
# model performance on unseen data

print(classification_report(y_test, y_pred_test))

In [ ]:
# classification matric on the test set 

conf = confusion_matrix(y_test, y_pred_test)
sns.heatmap(conf, annot=True, fmt="d", xticklabels=[0,1], yticklabels=[0,1])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Random Forest Confusion Matrix (Test Set)")
plt.show()

In [ ]:
# Feature importance

importances = pd.Series(rf_classifier.feature_importances_, index=X_train.columns)
importances.sort_values(ascending=False).head(15).plot(kind="bar", figsize=(10,5))
plt.title("Top 15 Feature Importances (Random Forest)")
plt.show()

#### The above shows the importance score of each feature. Ideally, it assists in getting an understanding of how a specific feature reduces uncertainty and makes good splits across all trees. The higher the feature's score metric, the more that feature was used in making a string, discriminative decision. 

## Second model that predicts where and when crashes are likely to occur, I chose to go this route to see if I can be able to zone in on the regions which will allow planners to get a heat map of risk thats more predictive and not just descriptive.


In [ ]:
# We should start of by making spatial coordinates into 1KM grid sections
grid_size = 0.01 
merged_df["lat_bin"] = (merged_df["latitude"] // grid_size) * grid_size
merged_df["lon_bin"] = (merged_df["longitude"] // grid_size) * grid_size

# Define the features 
X = merged_df[["lat_bin", "lon_bin", "crash_day_of_week", "crash_hour", "weather_condition", "lighting_condition"]]

# Defining the targets 
# We can also generate negative samples from random grid/time combos where crashes did not happen
all_combos = pd.MultiIndex.from_product([
    merged_df["lat_bin"].unique(),
    merged_df["lon_bin"].unique(),
    range(7),  # days of week
    range(24)  # hours
], names=["lat_bin", "lon_bin", "crash_day_of_week", "crash_hour"]) 


combo_df = pd.DataFrame(index=all_combos).reset_index()
combo_df = combo_df.merge(merged_df.groupby(["lat_bin", "lon_bin", "crash_day_of_week", "crash_hour"]).size().reset_index(name="crash_count"),
                          on=["lat_bin","lon_bin","crash_day_of_week","crash_hour"], how="left")

combo_df["crash_occurred"] = (combo_df["crash_count"].fillna(0) > 0).astype(int) 

# considering the features and targets
X = combo_df[["lat_bin","lon_bin","crash_day_of_week","crash_hour"]]
y = combo_df["crash_occurred"]




In [ ]:
# Encoding the categorical data
X = X.apply(LabelEncoder().fit_transform)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) 

#check shape
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Making the baseline model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Making evaluations on the model
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))

Considering that the ROC score here is at 99%, that signifies good separation and reason to consider that data leakage did not occur.

In [ ]:
print(f"The model's score on training dataset is: {model.score(X_train, y_train)*100:.2f}%")

In [ ]:
#accuracy on unseen data

print(f"The model's score on unseen data is: {accuracy_score(y_test, y_pred)*100:.2f}%")

In [ ]:
# Since the model seems to be fine, I will attempt to map the predictions where I will be taking the predicted probabilities and map them back

X_test["pred_prob"] = y_prob

plt.figure(figsize=(8,6))
plt.scatter(X_test["lon_bin"], X_test["lat_bin"], c=X_test["pred_prob"], cmap="Reds", s=50)
plt.colorbar(label="Predicted Crash Risk")
plt.title("Predicted Crash Hotspots in Chicago")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

The purpose of the above heatmap is to give predicted crash risk which city planners can use to make proper planning

### Now, I already tried making a fourth model, tried going with XGBoost and LightGBM and my PC could no longer handle either of them at this point. Which forces me to settle with what I have at the moment. Unfortunately, the same applies for hyperparemter tuning. Which will force me to go straight to storytelling 

# Storytelling 

## 1. Problem objective
#### Chicago experiences plenty of traffic crashes annually which ends up negatively impacting lives, property, and urban efficiency. As a way to best plan and reduce the crashes what can be done is to pinpoint where crashes cluster colliqually termed as black spots, and the factors that drive teh same, so that resources and the causes can be assessed to reduce accidents. Ideally, that is in line with the problem statement "To pinpoint high-risk locations and uncover the human, vehicle, and environmental conditions that are majorly associated to accidents."

## 2. Data Exploration (Key Insights)
#### From the analysis, most accidents happen during rush hour periods which is around 3PM to 7PM with the weekends having variations but a common factor being that they occur at night, but Saturdays being a constant figure with the highest number of crashes. While considering the crash severity, most crashes majorly involved property damage with a many having injuries and more interestingly for a 2million crash data low deaths which is actually commendable. Most crashes involve passenge type vehicles with motorcycle crashes and trucks having a low count but have significanlty higher severity rates. 

## 3. Patterns and Hotspots
#### Altough I'm not consident with my spatial interpretation, I will still do my best since it involves visual cues,it shouldnt be that difficult. Now, the crash density is not quite uniform with majority of it being concentrated in the dowtown regions with dense traffic and pedestrians, major intersection and arterial roads, and entertainment districts which majorly happends during weekends. 

## 4. Contributing Factors
#### Human factors involve situations where the driver was distracted while driving and alcohol related crashes spike at night, at the same time the you younger drivers are quite overrepresented in injury-causing crashes. Vehicle factors such as SUVs and Sedans dominate the number of high accidentc, and crashes involving motorcycles have the highest injury severity rate, Ironically when it comes to environmental factors, most crashes occur when the weather is mostly clear which asserts the notion that bad driving behaviour matters more than rain or snow. Also, poorly lit areas or intersections have significantly higher crash risks which was a very expected insight.

## 5. Predictive Modeling 
#### While descriptive analysis identifies past crash hotspots, we also explored whether machine learning can flag likely future hotspots based on location and crash attributes. Defined hotspots as the top 5% most crash-dense grid cells.
#### Features included time, location bins, vehicle type, driver attributes, weather, and lighting.
#### Tested models: Logistic Regression, XGBoost, LightGBM (baseline implementation).
#### Result:
#### Models reached ~85–90% accuracy, but struggled with precision on rare hotspots (too few positives compared to negatives).
#### Interpretation: Predicting exact hotspot cells is difficult without richer urban features (traffic flow, road geometry, enforcement data).

## 6. Recommendations & Next Steps
#### The data highlights clear interventions for the city and for further research.

#### City-Level Recommendations:

#### Target Enforcement & Awareness

#### Focus traffic enforcement during evening rush hours and weekend nights.

#### Prioritize young drivers with awareness campaigns.

#### Infrastructure Improvements

#### Invest in better lighting at crash-prone intersections.

#### Redesign top hotspot intersections (traffic calming, pedestrian protection).

#### Data-Driven Policy

#### Require ride-share & delivery services to share driving data (these fleets may be overrepresented).

#### Consider stricter penalties for night-time alcohol-related driving.

#### Analytical Next Steps:

#### Integrate road network data (speed limits, lane counts).

#### Include traffic volume data (exposure-adjusted risk, not just raw crash counts).

#### Expand prediction work with deep learning spatial models if data allows.